In [10]:
from mxnet import np, npx
from mxnet.gluon import nn

npx.set_np()

# K = np.array([[1.0, -1.0]])

# X = np.ones((6, 8))


# # 为了方便起见，我们定义了一个计算卷积层的函数。
# # 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
# def comp_conv2d(conv2d, X):
#     conv2d.initialize()
#     X = X.reshape((1, 1) + X.shape)# 这里的（1，1）表示批量大小和通道数都是1
#     Y = conv2d(X)# 省略前两个维度：批量大小和通道
#     return Y.reshape(Y.shape[2:])

# # 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
# conv2d = nn.Conv2D(1, kernel_size=3, padding=1)
# X = np.random.uniform(size=(8, 8))
# print(comp_conv2d(conv2d, X).shape)
# # print(conv2d)

# conv2d = nn.Conv2D(1, kernel_size=(5, 3), padding=(2, 1))
# # print(conv2d)
# print(comp_conv2d(conv2d, X).shape)


def corr2d(X, K):  #@save
    """计算二维互相关运算"""
    h, w = K.shape
    Y = np.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y


def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(corr2d(x, k) for x, k in zip(X, K))

X = np.array([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])

K = np.array([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
 
    
print(corr2d_multi_in(X, K))

def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return np.stack([corr2d_multi_in(X, k) for k in K], 0)

def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape #3,3,3
    c_o = K.shape[0] #2
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = np.dot(K, X)
    return Y.reshape((c_o, h, w))

X = np.random.normal(0, 1, (3, 3, 3))
K = np.random.normal(0, 1, (2, 3, 1, 1))
 

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

print(Y1)

assert float(np.abs(Y1 - Y2).sum()) < 1e-6

 



[[ 56.  72.]
 [104. 120.]]
[[[-1.7858424   0.506384   -0.2884437 ]
  [ 0.13996238  1.4861625  -0.1357854 ]
  [ 0.40117115  0.60125387  4.1696186 ]]

 [[ 0.06462019  0.07530563  0.01569504]
  [-0.27643713 -1.8316431  -0.43535706]
  [-0.5620969  -0.35807738 -2.0872386 ]]]
[[[-1.7858424   0.5063841  -0.2884437 ]
  [ 0.13996238  1.4861625  -0.1357854 ]
  [ 0.40117112  0.60125387  4.1696186 ]]

 [[ 0.06462014  0.07530563  0.01569504]
  [-0.27643716 -1.8316432  -0.4353571 ]
  [-0.5620969  -0.35807735 -2.0872386 ]]]
[[[0.0000000e+00 5.9604645e-08 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [2.9802322e-08 0.0000000e+00 0.0000000e+00]]

 [[5.2154064e-08 0.0000000e+00 5.5879354e-09]
  [2.9802322e-08 1.1920929e-07 2.9802322e-08]
  [0.0000000e+00 2.9802322e-08 0.0000000e+00]]]
